## Data Cleaning

#### Integrate data from multiple sources and transform into a large set of features

In [ ]:
import pandas as pd
import pickle
import calendar
import re

from itertools import islice

precipitation_values_list = [" Drizzle.", " No Precipitation.", " Rain."]

usa_federal_holidays = {
    '2015': ['1/1', '1/19', '2/16', '5/25', '7/4', '9/7', '10/12', '11/11' '11/26', '12/25'],
    '2016': ['1/1', '1/18', '2/15', '5/30', '7/4', '9/5', '10/10', '11/11', '11/24', '12/26'],
    '2017': ['1/1', '1/2', '1/16', '2/20', '5/29', '7/4', '9/4', '10/9', '11/10', '11/11', '11/23', '12/25'],
    '2018': ['1/1', '1/15', '2/19', '5/28', '7/4', '9/3', '10/8', '11/11', '11/12', '11/22', '12/25'],
    '2019': ['1/1', '1/21', '2/18', '5/27', '7/4', '9/2', '10/14', '11/11', '11/28', '12/25'],
    '2020': ['1/1', '1/20', '2/17', '5/25', '7/3', '7/4', '9/7', '10/12', '11/11', '11/26', '12/25'],
    '2021': ['1/1', '1/18', '2/15', '5/31', '6/18', '6/19', '7/4', '7/5', '9/6', '10/11', '11/11', '11/25', '12/24', '12/25', '12/31'],
    '2022': ['1/1', '1/17', '2/21', '5/30', '6/19', '6/20', '7/4', '9/5', '10/10', '11/11', '11/24', '12/25', '12/26']
}

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

def flatten_games(games, year):
    games_flatten = {}
    holiday_list = usa_federal_holidays[year]

    # create a dictionary, key is unique team name, value is their streak which is default to 0
    teams_streak = {}
    unique_team_names = set()
    for game_id in games.keys():
        # get team1 name and team2 name and add them to unique_team_names
        unique_team_names.add(games[game_id]['team1']['name'])
        unique_team_names.add(games[game_id]['team2']['name'])
    
    # add unique team names to teams_streak
    for team_name in unique_team_names:
        teams_streak[team_name] = {
            'pre_win': 0,
            'pre_loss': 0,
            'cumulative_streak': 0
        }

    for game_id in games.keys():
        games_flatten[game_id] = {}
        games_flatten[game_id]['game_page_url'] = game_id
        games_flatten[game_id]['have_att'] = games[game_id]['meta_game_info']['have_att']
        games_flatten[game_id]['start_time'] = games[game_id]['meta_game_info']['start_time']
        
        # default value
        day = 1
        month = 1
        try:
            day = calendar.day_name[games[game_id]['meta_game_info']['start_time'].weekday()]
            month = calendar.month_name[games[game_id]['meta_game_info']['start_time'].month]
        except:
            # game_id is the url of the game page and we can use regex to get numeric part of the url and use it as day and month
            # example: https://www.baseball-reference.com/boxes/CHN/CHN201904010.shtml
            # get numeric part of the url
            numeric_part = re.findall(r'\d+', game_id)
            # get day and month
            # get substring of the numeric part
            day = int(numeric_part[0][6:8])
            month = int(numeric_part[0][4:6])

        date_str = f"{month}/{day}"
        games_flatten[game_id]['day'] = day
        games_flatten[game_id]['month'] = month
        games_flatten[game_id]['is_federal_holiday'] = 1 if date_str in holiday_list else 0
        games_flatten[game_id]['duration'] = games[game_id]['meta_game_info']['duration']
        games_flatten[game_id]['venue'] = games[game_id]['meta_game_info']['venue']
        games_flatten[game_id]['at_night'] = games[game_id]['meta_game_info']['at_night']
        games_flatten[game_id]['on_grass'] = games[game_id]['meta_game_info']['on_grass']
        games_flatten[game_id]['weather_description'] = games[game_id]['meta_game_info']['Start Time Weather']
        
        # default value for weather factors
        temperature = 0
        wind_speed = 0
        weather = 'Sunny'
        precipitation = 0

        try:
            temperature = int(re.findall(r'\d+', games[game_id]['meta_game_info']['Start Time Weather'].split(',')[0])[0])
            # use regex to get wind speed, which is only number in the string
            wind_speed = int(re.findall(r'\d+', games[game_id]['meta_game_info']['Start Time Weather'].split(',')[1])[0])
            weather = (games[game_id]['meta_game_info']['Start Time Weather'].split(',')[2]).strip()
        except:
            print(f"cannot get temperature and wind_speed from weather description: {games[game_id]['meta_game_info']['Start Time Weather']}, automatically set to 0")
        
        # get comma count in Start Time Weather
        comma_count = games[game_id]['meta_game_info']['Start Time Weather'].count(',')

        if comma_count < 3:
            precipitation = 4
        else:
            precipitation_str = games[game_id]['meta_game_info']['Start Time Weather'].split(',')[3]
            match_index = -1
            for i in range(len(precipitation_values_list)):
                if precipitation_values_list[i] == precipitation_str:
                    match_index = i
                    break
            precipitation = match_index
            # ensure precipitation is in range 0 to 3
            if precipitation == -1:
                # something wrong with precipitation, use input to get precipitation
                print(f"cannot get precipitation from weather description: {games[game_id]['meta_game_info']['Start Time Weather']}")
                precipitation = input("please input precipitation: ")

        games_flatten[game_id]['temperature'] = temperature
        games_flatten[game_id]['wind_speed'] = wind_speed
        games_flatten[game_id]['weather'] = weather
        games_flatten[game_id]['precipitation'] = precipitation
        games_flatten[game_id]['umpires'] = games[game_id]['meta_game_info']['Umpires']
        games_flatten[game_id]['attendance'] = games[game_id]['meta_game_info']['att']
        games_flatten[game_id]['team1_name'] = games[game_id]['team1']['name']
        games_flatten[game_id]['team1_pre_win'] = games[game_id]['team1']['pre_win']
        games_flatten[game_id]['team1_pre_loss'] = games[game_id]['team1']['pre_loss']
        # win_pct = win / (win + loss)
        if games[game_id]['team1']['pre_win'] + games[game_id]['team1']['pre_loss'] == 0:
            games_flatten[game_id]['team1_pre_win_pct'] = 0
        else:
            games_flatten[game_id]['team1_pre_win_pct'] = games[game_id]['team1']['pre_win'] / (games[game_id]['team1']['pre_win'] + games[game_id]['team1']['pre_loss'])
        games_flatten[game_id]['team1_players'] = games[game_id]['team1']['player_df']

        if games[game_id]['team1']['pre_win'] > teams_streak[games[game_id]['team1']['name']]['pre_win']:
            if teams_streak[games[game_id]['team1']['name']]['cumulative_streak'] < 0:
                teams_streak[games[game_id]['team1']['name']]['cumulative_streak'] = 1
            else:
                teams_streak[games[game_id]['team1']['name']]['cumulative_streak'] += 1
            games_flatten[game_id]['team1_streak'] = teams_streak[games[game_id]['team1']['name']]['cumulative_streak']
        elif games[game_id]['team1']['pre_loss'] > teams_streak[games[game_id]['team1']['name']]['pre_loss']:
            if teams_streak[games[game_id]['team1']['name']]['cumulative_streak'] > 0:
                teams_streak[games[game_id]['team1']['name']]['cumulative_streak'] = -1
            else:
                teams_streak[games[game_id]['team1']['name']]['cumulative_streak'] -= 1
            games_flatten[game_id]['team1_streak'] = teams_streak[games[game_id]['team1']['name']]['cumulative_streak']
        else:
            games_flatten[game_id]['team1_streak'] = 0

        teams_streak[games[game_id]['team1']['name']]['pre_win'] = games[game_id]['team1']['pre_win']
        teams_streak[games[game_id]['team1']['name']]['pre_loss'] = games[game_id]['team1']['pre_loss']
        
        # for team2 do the same thing
        games_flatten[game_id]['team2_name'] = games[game_id]['team2']['name']
        games_flatten[game_id]['team2_pre_win'] = games[game_id]['team2']['pre_win']
        games_flatten[game_id]['team2_pre_loss'] = games[game_id]['team2']['pre_loss']
        if games[game_id]['team2']['pre_win'] + games[game_id]['team2']['pre_loss'] == 0:
            games_flatten[game_id]['team2_pre_win_pct'] = 0
        else:
            games_flatten[game_id]['team2_pre_win_pct'] = games[game_id]['team2']['pre_win'] / (games[game_id]['team2']['pre_win'] + games[game_id]['team2']['pre_loss'])
        games_flatten[game_id]['team2_players'] = games[game_id]['team2']['player_df']

        if games[game_id]['team2']['pre_win'] > teams_streak[games[game_id]['team2']['name']]['pre_win']:
            if teams_streak[games[game_id]['team2']['name']]['cumulative_streak'] < 0:
                teams_streak[games[game_id]['team2']['name']]['cumulative_streak'] = 1
            else:
                teams_streak[games[game_id]['team2']['name']]['cumulative_streak'] += 1
            games_flatten[game_id]['team2_streak'] = teams_streak[games[game_id]['team2']['name']]['cumulative_streak']
        elif games[game_id]['team2']['pre_loss'] > teams_streak[games[game_id]['team2']['name']]['pre_loss']:
            if teams_streak[games[game_id]['team2']['name']]['cumulative_streak'] > 0:
                teams_streak[games[game_id]['team2']['name']]['cumulative_streak'] = -1
            else:
                teams_streak[games[game_id]['team2']['name']]['cumulative_streak'] -= 1
            games_flatten[game_id]['team2_streak'] = teams_streak[games[game_id]['team2']['name']]['cumulative_streak']
        else:
            games_flatten[game_id]['team2_streak'] = 0

        teams_streak[games[game_id]['team2']['name']]['pre_win'] = games[game_id]['team2']['pre_win']
        teams_streak[games[game_id]['team2']['name']]['pre_loss'] = games[game_id]['team2']['pre_loss']

    return games_flatten

In [ ]:
games = {}
for year in range(2015, 2022):
    print(year)
    if year == 2020 or year == 2021:
        continue
    cur_year_games = pd.read_pickle(f"gamesData{year}.pickle")
    cur_year_flatten_games = flatten_games(cur_year_games, str(year))
    games.update(cur_year_flatten_games)
# print out the total number of games
print(f"Total number of games: {len(games)}")
# save the games to a pickle file
with open("trainset.pickle", "wb") as f:
    pickle.dump(games, f)

# for 2022
games = {}
cur_year_games = pd.read_pickle(f"gamesData2022.pickle")
cur_year_flatten_games = flatten_games(cur_year_games, '2022')
games.update(cur_year_flatten_games)
with open("testset.pickle", "wb") as f:
    pickle.dump(games, f)
print("Done!")

In [2]:
trainset = pd.read_csv("processed_training.csv")
testset = pd.read_csv("processed_test.csv")

# take a look at the data
print(f"trainset have {trainset.shape[0]} rows and {trainset.shape[1]} columns")
print(f"testset have {testset.shape[0]} rows and {testset.shape[1]} columns")

trainset have 12270 rows and 219 columns
testset have 2445 rows and 219 columns


In [8]:
trainset.head()

,attendance,is_federal_holiday,venue,on_grass,temperature,wind_speed,team1_name,team1_pre_win,team1_pre_loss,team1_pre_win_pct,...,season_type,season,home_team_avg_att_last_year,start_hour,start_hour_label_afternoon,start_hour_label_evening,start_hour_label_night,start_hour_label_noon,game_page_url,start_time
0,35055,0,Wrigley Field,1,-2.786672,-0.093090,STL,-1.62645,-1.654514,0.0,...,regular,2015,0.345307,19,0,1,0,0,https://www.baseball-reference.com/boxes/CHN/C...,2015-04-05 19:17:00
1,49043,0,Chase Field,1,0.705205,-0.093090,SFG,-1.62645,-1.654514,0.0,...,regular,2015,-0.573314,19,0,1,0,0,https://www.baseball-reference.com/boxes/ARI/A...,2015-04-06 19:13:00
2,43633,0,Great American Ball Park,1,-0.521670,1.490839,PIT,-1.62645,-1.654514,0.0,...,regular,2015,0.066649,16,1,0,0,0,https://www.baseball-reference.com/boxes/CIN/C...,2015-04-06 16:10:00
3,45030,0,Comerica Park,1,-2.031671,-0.489072,MIN,-1.62645,-1.654514,0.0,...,regular,2015,0.766348,13,0,0,0,1,https://www.baseball-reference.com/boxes/DET/D...,2015-04-06 13:08:00
4,43753,0,Minute Maid Park,1,0.516455,1.292848,CLE,-1.62645,-1.654514,0.0,...,regular,2015,-1.084575,18,0,1,0,0,https://www.baseball-reference.com/boxes/HOU/H...,2015-04-06 18:10:00


#### get dummies for categorical variables

##### check day of week values

In [9]:
# check day and month value for trainset
print(f"unique day value: {trainset['day'].unique()}")
print(trainset['day'].value_counts())

unique day value: ['Sunday' 'Monday' 'Tuesday' 'Wednesday' 'Thursday' 'Friday' 'Saturday']
Saturday     2010
Sunday       1988
Friday       1960
Tuesday      1863
Wednesday    1853
Thursday     1299
Monday       1297
Name: day, dtype: int64


In [10]:
# check day and month value for testset
print(f"unique day value: {testset['day'].unique()}")
print(testset['day'].value_counts())

unique day value: ['Thursday' 'Friday' 'Saturday' 'Sunday' 'Monday' 'Tuesday' 'Wednesday']
Saturday     418
Sunday       389
Tuesday      388
Friday       380
Wednesday    367
Thursday     258
Monday       245
Name: day, dtype: int64


##### check month values

In [26]:
# check month value for trainset
print(f"unique month value: {trainset['month'].unique()}")
print(trainset['month'].value_counts())

unique month value: ['April' 'May' 'June' 'July' 'August' 'September' 'October' 'November'
 'March']
May          2090
August       2090
June         2022
September    2021
July         1868
April        1806
October       277
March          92
November        4
Name: month, dtype: int64


In [27]:
# check month value for testset
print(f"unique month value: {testset['month'].unique()}")
print(testset['month'].value_counts())

unique month value: ['April' 'May' 'June' 'July' 'August' 'September' 'October' 'November']
August       418
May          409
June         402
September    397
July         389
April        315
October      111
November       4
Name: month, dtype: int64


##### check weather values

In [28]:
# check weather values for trainset
print(f"unique weather value: {trainset['weather'].unique()}")
print(trainset['weather'].value_counts())

unique weather value: ['Sunny' 'Overcast' 'Cloudy' 'Drizzle' 'In Dome' 'Rain']
Sunny       5290
Cloudy      4252
In Dome     1823
Overcast     776
Drizzle       71
Rain          58
Name: weather, dtype: int64


In [29]:
# check weather values for testset
print(f"unique weather value: {testset['weather'].unique()}")
print(testset['weather'].value_counts())

unique weather value: ['Sunny' 'Cloudy' 'In Dome' 'Overcast']
Cloudy      1024
Sunny        804
In Dome      448
Overcast     169
Name: weather, dtype: int64


In [30]:
##### get dummies for these checked columns
trainset = pd.get_dummies(trainset, columns=['day', 'month', 'weather'])
testset = pd.get_dummies(testset, columns=['day', 'month', 'weather'])


#### make true / false columns to 1 / 0

In [46]:
# do lambda transformation for the target variable
trainset['at_night'] = trainset['at_night'].apply(lambda x: 1 if x else 0)
testset['at_night'] = testset['at_night'].apply(lambda x: 1 if x else 0)

trainset['on_grass'] = trainset['on_grass'].apply(lambda x: 1 if x else 0)
testset['on_grass'] = testset['on_grass'].apply(lambda x: 1 if x else 0)

In [88]:
new_trainset = pd.read_csv("processed_training.csv")
new_trainset.head(10)

,attendance,is_federal_holiday,venue,at_night,on_grass,temperature,wind_speed,team1_name,team1_pre_win,team1_pre_loss,...,month_May,month_November,month_October,month_September,weather_Cloudy,weather_Drizzle,weather_In Dome,weather_Overcast,weather_Rain,weather_Sunny
0,35055,0,Wrigley Field,1,1,44,7,StLouisCardinals,0,0,...,0,0,0,0,0,0,0,0,0,1
1,49043,0,Chase Field,1,1,81,7,SanFranciscoGiants,0,0,...,0,0,0,0,0,0,0,0,0,1
2,43633,0,Great American Ball Park,0,1,68,15,PittsburghPirates,0,0,...,0,0,0,0,0,0,0,1,0,0
3,45030,0,Comerica Park,0,1,52,5,MinnesotaTwins,0,0,...,0,0,0,0,1,0,0,0,0,0
4,43753,0,Minute Maid Park,1,1,79,14,ClevelandIndians,0,0,...,0,0,0,0,1,0,0,0,0,0
5,40085,0,Kauffman Stadium,0,1,59,6,ChicagoWhiteSox,0,0,...,0,0,0,0,0,1,0,0,0,0
6,53518,0,Dodger Stadium,0,1,65,1,SanDiegoPadres,0,0,...,0,0,0,0,1,0,0,0,0,0
7,36969,0,Marlins Park,0,1,80,16,AtlantaBraves,0,0,...,0,0,0,0,1,0,0,0,0,0
8,46032,0,Miller Park,0,1,70,0,ColoradoRockies,0,0,...,0,0,0,0,0,0,1,0,0,0
9,48469,0,Yankee Stadium III,0,1,54,5,TorontoBlueJays,0,0,...,0,0,0,0,1,0,0,0,0,0


In [89]:
new_trainset['team1_name'].unique()

array(['StLouisCardinals', 'SanFranciscoGiants', 'PittsburghPirates',
       'MinnesotaTwins', 'ClevelandIndians', 'ChicagoWhiteSox',
       'SanDiegoPadres', 'AtlantaBraves', 'ColoradoRockies',
       'TorontoBlueJays', 'TexasRangers', 'BostonRedSox',
       'LosAngelesAngelsofAnaheim', 'BaltimoreOrioles', 'NewYorkMets',
       'KansasCityRoyals', 'LosAngelesDodgers', 'DetroitTigers',
       'ChicagoCubs', 'TampaBayRays', 'SeattleMariners',
       'WashingtonNationals', 'HoustonAstros', 'MiamiMarlins',
       'NewYorkYankees', 'CincinnatiReds', 'OaklandAthletics',
       'PhiladelphiaPhillies', 'ArizonaDiamondbacks', 'MilwaukeeBrewers',
       'LosAngelesAngels'], dtype=object)

trainset team name that should be modified:

* ClevelandIndians -> ClevelandGuardians
* LosAngelesAngelsofAnaheim -> LosAngelesAngels

In [90]:
def change_name(name):
    if name == 'ClevelandIndians':
        return 'ClevelandGuardians'
    elif name == 'LosAngelesAngelsofAnaheim':
        return 'LosAngelesAngels'
    else:
        return name
new_trainset['team1_name'] = new_trainset['team1_name'].apply(lambda x: change_name(x))
new_trainset['team2_name'] = new_trainset['team2_name'].apply(lambda x: change_name(x))

In [91]:
new_trainset['team1_name'].unique()

array(['StLouisCardinals', 'SanFranciscoGiants', 'PittsburghPirates',
       'MinnesotaTwins', 'ClevelandGuardians', 'ChicagoWhiteSox',
       'SanDiegoPadres', 'AtlantaBraves', 'ColoradoRockies',
       'TorontoBlueJays', 'TexasRangers', 'BostonRedSox',
       'LosAngelesAngels', 'BaltimoreOrioles', 'NewYorkMets',
       'KansasCityRoyals', 'LosAngelesDodgers', 'DetroitTigers',
       'ChicagoCubs', 'TampaBayRays', 'SeattleMariners',
       'WashingtonNationals', 'HoustonAstros', 'MiamiMarlins',
       'NewYorkYankees', 'CincinnatiReds', 'OaklandAthletics',
       'PhiladelphiaPhillies', 'ArizonaDiamondbacks', 'MilwaukeeBrewers'],
      dtype=object)

In [93]:
# make sure team names in trainset has no difference with team names in testset
teamnames_train = set(new_trainset['team1_name'].unique())
teamnames_test = set(testset['team1_name'].unique())

print(teamnames_train - teamnames_test)

set()


#### merge season type and train test set

In [94]:
season_type_train = pd.read_csv("season_type.csv")
season_type_test = pd.read_csv("season_type_test.csv")

In [95]:
# trainset should append season_type
new_testset = pd.read_csv("processed_test.csv")
new_trainset['season_type'] = season_type_train['season']
new_testset['season_type'] = season_type_test['season']

In [97]:
new_trainset.to_csv("processed_training.csv", index=False)
new_testset.to_csv("processed_test.csv", index=False)

#### make every game can have a information about the average attendance of the home team in last season

##### first need to do a mapping dictionary for team abbreviation and team name

In [105]:
stadium = pd.read_csv("../data/stadium/avg_attendance_2014.csv")

team_abbr = set(stadium['Team'])

In [99]:
new_trainset['team1_name'].unique()

array(['StLouisCardinals', 'SanFranciscoGiants', 'PittsburghPirates',
       'MinnesotaTwins', 'ClevelandGuardians', 'ChicagoWhiteSox',
       'SanDiegoPadres', 'AtlantaBraves', 'ColoradoRockies',
       'TorontoBlueJays', 'TexasRangers', 'BostonRedSox',
       'LosAngelesAngels', 'BaltimoreOrioles', 'NewYorkMets',
       'KansasCityRoyals', 'LosAngelesDodgers', 'DetroitTigers',
       'ChicagoCubs', 'TampaBayRays', 'SeattleMariners',
       'WashingtonNationals', 'HoustonAstros', 'MiamiMarlins',
       'NewYorkYankees', 'CincinnatiReds', 'OaklandAthletics',
       'PhiladelphiaPhillies', 'ArizonaDiamondbacks', 'MilwaukeeBrewers'],
      dtype=object)

In [110]:
# now team_abbr is a list of MLB team abbreviations, we need to make a dictionary to map team name to team abbreviation
team_abbr_map = {
    'ArizonaDiamondbacks': 'ARI',
    'AtlantaBraves': 'ATL',
    'BaltimoreOrioles': 'BAL',
    'BostonRedSox': 'BOS',
    'ChicagoCubs': 'CHC',
    'ChicagoWhiteSox': 'CHW',
    'CincinnatiReds': 'CIN',
    'ClevelandGuardians': 'CLE',
    'ColoradoRockies': 'COL',
    'DetroitTigers': 'DET',
    'HoustonAstros': 'HOU',
    'KansasCityRoyals': 'KCR',
    'LosAngelesAngels': 'LAA',
    'LosAngelesDodgers': 'LAD',
    'MiamiMarlins': 'MIA',
    'MilwaukeeBrewers': 'MIL',
    'MinnesotaTwins': 'MIN',
    'NewYorkMets': 'NYM',
    'NewYorkYankees': 'NYY',
    'OaklandAthletics': 'OAK',
    'PhiladelphiaPhillies': 'PHI',
    'PittsburghPirates': 'PIT',
    'SanDiegoPadres': 'SDP',
    'SanFranciscoGiants': 'SFG',
    'SeattleMariners': 'SEA',
    'StLouisCardinals': 'STL',
    'TampaBayRays': 'TBR',
    'TexasRangers': 'TEX',
    'TorontoBlueJays': 'TOR',
    'WashingtonNationals': 'WSN'
}

##### make sure the team name has correctly mapped

In [112]:
for abbr in team_abbr:
    if abbr not in team_abbr_map.values():
        print(abbr)
    
print("------")
for dict_value in team_abbr_map.values():
    if dict_value not in team_abbr:
        print(dict_value)

------


##### read train / test set and change team name to team abbreviation

In [113]:
trainset = pd.read_csv("processed_training.csv")
testset = pd.read_csv("processed_test.csv")

In [115]:
# change team name to team abbreviation
def change_name(name):
    if name not in team_abbr_map.keys():
        print(f"{name} is not in the dictionary, please check")
        return name
    return team_abbr_map[name]

trainset['team1_name'] = trainset['team1_name'].apply(lambda x: change_name(x))
trainset['team2_name'] = trainset['team2_name'].apply(lambda x: change_name(x))

testset['team1_name'] = testset['team1_name'].apply(lambda x: change_name(x))
testset['team2_name'] = testset['team2_name'].apply(lambda x: change_name(x))

##### save csv

In [ ]:
trainset.to_csv("processed_training.csv", index=False)
testset.to_csv("processed_test.csv", index=False)

#### split trainset according to season, it starts from 2015, so we need to split it into 2015-2016, 2016-2017, 2017-2018, 2018-2019

In [128]:
trainset = pd.read_csv("processed_training.csv")

season_games_idx = [2452, 4912, 7367, 9816]

season_list = []
for i in range(trainset.shape[0]):
    if i < 2452:
        season_list.append(2015)
    elif i < 4912:
        season_list.append(2016)
    elif i < 7367:
        season_list.append(2017)
    elif i < 9816:
        season_list.append(2018)
    else:
        season_list.append(2019)

trainset['season'] = season_list

In [129]:
avg_att_2014 = pd.read_csv("../data/stadium/avg_attendance_2014.csv")
avg_att_2015 = pd.read_csv("../data/stadium/avg_attendance_2015.csv")
avg_att_2016 = pd.read_csv("../data/stadium/avg_attendance_2016.csv")
avg_att_2017 = pd.read_csv("../data/stadium/avg_attendance_2017.csv")
avg_att_2018 = pd.read_csv("../data/stadium/avg_attendance_2018.csv")
avg_att_2019 = pd.read_csv("../data/stadium/avg_attendance_2019.csv")

In [130]:
avg_att_list = []
for i in range(trainset.shape[0]):
    team_abbr = trainset.iloc[i]['team2_name']
    if trainset.iloc[i]['season'] == 2015:
        avg_att_list.append(avg_att_2014[avg_att_2014['Team'] == team_abbr]['Avg Attendance'].values[0])
    elif trainset.iloc[i]['season'] == 2016:
        avg_att_list.append(avg_att_2015[avg_att_2015['Team'] == team_abbr]['Avg Attendance'].values[0])
    elif trainset.iloc[i]['season'] == 2017:
        avg_att_list.append(avg_att_2016[avg_att_2016['Team'] == team_abbr]['Avg Attendance'].values[0])
    elif trainset.iloc[i]['season'] == 2018:
        avg_att_list.append(avg_att_2017[avg_att_2017['Team'] == team_abbr]['Avg Attendance'].values[0])
    elif trainset.iloc[i]['season'] == 2019:
        avg_att_list.append(avg_att_2018[avg_att_2018['Team'] == team_abbr]['Avg Attendance'].values[0])

trainset['home_team_avg_att_last_year'] = avg_att_list

In [131]:
# for testset, we need to use 2019 avg attendance
avg_att_list = []
for i in range(testset.shape[0]):
    team_abbr = testset.iloc[i]['team2_name']
    avg_att_list.append(avg_att_2019[avg_att_2019['Team'] == team_abbr]['Avg Attendance'].values[0])

testset['home_team_avg_att_last_year'] = avg_att_list

In [132]:
trainset['home_team_avg_att_last_year'].value_counts()

36796.901235    91
28476.827160    90
46492.049383    90
24154.098765    89
17806.474359    89
                ..
25042.271605    78
21405.212500    78
21828.662500    78
20110.212500    77
30805.912500    75
Name: home_team_avg_att_last_year, Length: 150, dtype: int64

In [133]:
testset['home_team_avg_att_last_year'].value_counts()

35276.135802    88
33671.864198    88
29585.172840    85
21606.716049    83
32776.790123    83
42967.814815    83
49065.543210    83
41827.898734    83
35993.851852    81
10016.074074    81
18412.827160    81
21425.649351    81
26333.259259    81
26364.320988    81
37321.135802    81
14734.187500    81
33429.135802    81
27898.530864    81
38208.209877    81
22112.456790    81
28435.790123    80
22334.259259    80
18767.875000    80
36953.629630    80
36090.530864    80
18495.737500    80
22008.126582    80
30531.650000    80
20626.345679    79
16347.587500    78
Name: home_team_avg_att_last_year, dtype: int64

In [135]:
# save the processed training and test set
trainset.to_csv("processed_training.csv", index=False)
testset.to_csv("processed_test.csv", index=False)

#### standardize the data

In [136]:
trainset.head(10)

,attendance,is_federal_holiday,venue,at_night,on_grass,temperature,wind_speed,team1_name,team1_pre_win,team1_pre_loss,...,month_September,weather_Cloudy,weather_Drizzle,weather_In Dome,weather_Overcast,weather_Rain,weather_Sunny,season_type,season,home_team_avg_att_last_year
0,35055,0,Wrigley Field,1,1,44,7,STL,0,0,...,0,0,0,0,0,0,1,regular,2015,32742.135802
1,49043,0,Chase Field,1,1,81,7,SFG,0,0,...,0,0,0,0,0,0,1,regular,2015,25601.604938
2,43633,0,Great American Ball Park,0,1,68,15,PIT,0,0,...,0,0,0,0,1,0,0,regular,2015,30576.098765
3,45030,0,Comerica Park,0,1,52,5,MIN,0,0,...,0,1,0,0,0,0,0,regular,2015,36014.925926
4,43753,0,Minute Maid Park,1,1,79,14,CLE,0,0,...,0,1,0,0,0,0,0,regular,2015,21627.518519
5,40085,0,Kauffman Stadium,0,1,59,6,CHW,0,0,...,0,0,1,0,0,0,0,regular,2015,24154.098765
6,53518,0,Dodger Stadium,0,1,65,1,SDP,0,0,...,0,1,0,0,0,0,0,regular,2015,46695.518519
7,36969,0,Marlins Park,0,1,80,16,ATL,0,0,...,0,1,0,0,0,0,0,regular,2015,21386.209877
8,46032,0,Miller Park,0,1,70,0,COL,0,0,...,0,0,0,1,0,0,0,regular,2015,34535.604938
9,48469,0,Yankee Stadium III,0,1,54,5,TOR,0,0,...,0,1,0,0,0,0,0,regular,2015,42520.300000


##### columns that need to be standardized

* temperature
* wind_speed
* team1_pre_win,
* team1_pre_loss,
* team1_streak,
* team2_pre_win,
* team2_pre_loss,
* team2_streak,
* start_hour
* salary-500-800
* salary-800-1500
* salary-1500'

In [142]:
# standardize the numerical features
# these features include:
"""
* temperature
* wind_speed
* team1_pre_win,
* team1_pre_loss,
* team1_streak,
* team2_pre_win,
* team2_pre_loss,
* team2_streak,
* start_hour
* salary-500-800
* salary-800-1500
* salary-1500'
* avg_att_last_year
"""
numerical_columns = [
    'temperature', 
    'wind_speed', 
    'team1_pre_win', 
    'team1_pre_loss', 
    'team1_streak', 
    'team2_pre_win', 
    'team2_pre_loss', 
    'team2_streak', 
    'start_hour', 
    'salary-500-800', 
    'salary-800-1500', 
    'salary-1500',
    'home_team_avg_att_last_year'
]
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
trainset[numerical_columns] = scaler.fit_transform(trainset[numerical_columns])
testset[numerical_columns] = scaler.transform(testset[numerical_columns])

In [143]:
trainset[numerical_columns].describe()

,temperature,wind_speed,team1_pre_win,team1_pre_loss,team1_streak,team2_pre_win,team2_pre_loss,team2_streak,start_hour,salary-500-800,salary-800-1500,salary-1500,home_team_avg_att_last_year
count,1.227000e+04,1.227000e+04,1.227000e+04,1.227000e+04,1.227000e+04,1.227000e+04,1.227000e+04,1.227000e+04,1.227000e+04,1.227000e+04,1.227000e+04,1.227000e+04,1.227000e+04
mean,-5.559259e-17,6.022530e-17,-6.485802e-17,6.485802e-17,-1.158179e-17,-1.389815e-16,-9.265431e-17,2.779629e-17,-4.933842e-16,-1.760432e-16,-7.412345e-17,-1.853086e-17,-1.111852e-16
std,1.000041e+00,1.000041e+00,1.000041e+00,1.000041e+00,1.000041e+00,1.000041e+00,1.000041e+00,1.000041e+00,1.000041e+00,1.000041e+00,1.000041e+00,1.000041e+00,1.000041e+00
min,-4.391048e+00,-1.479027e+00,-1.626450e+00,-1.654514e+00,-5.013797e+00,-1.624298e+00,-1.656594e+00,-5.158302e+00,-2.221342e+00,-2.188773e+00,-2.379852e+00,-2.240027e+00,-2.578688e+00
25%,-5.216702e-01,-6.870628e-01,-8.342849e-01,-8.388963e-01,-7.649072e-01,-8.324987e-01,-8.405738e-01,-4.502163e-01,-1.481778e+00,-7.276858e-01,-6.505367e-01,-7.054449e-01,-7.163250e-01
50%,4.458010e-02,-9.308968e-02,-4.211969e-02,-2.327867e-02,-3.786445e-01,-4.069988e-02,-2.455376e-02,3.344647e-01,7.369111e-01,-2.406567e-01,-2.182080e-01,6.184602e-02,-2.413237e-02
75%,7.052055e-01,6.988745e-01,7.500456e-01,7.923390e-01,3.938809e-01,7.510990e-01,7.914663e-01,7.268052e-01,7.369111e-01,7.334015e-01,6.464495e-01,8.291369e-01,7.670928e-01
max,3.253332e+00,4.064722e+00,2.651242e+00,2.953726e+00,5.415296e+00,2.651416e+00,3.035522e+00,7.788934e+00,1.846256e+00,4.629634e+00,3.672751e+00,3.898301e+00,2.185056e+00


##### start_hour

can be categorized into 3 levels:
* noon: 11, 12, 13
* afternoon: 14, 15, 16
* evening: 17, 18, 19
* night: 20, 21, 22

In [144]:
# first drop at_night column
trainset.drop(columns=['at_night'], inplace=True)
testset.drop(columns=['at_night'], inplace=True)

In [5]:
start_hour_levels_train = []
for i in range(trainset.shape[0]):
    if int(trainset.iloc[i]['start_hour']) < 14:
        start_hour_levels_train.append("noon")
    elif int(trainset.iloc[i]['start_hour']) < 17:
        start_hour_levels_train.append("afternoon")
    elif int(trainset.iloc[i]['start_hour']) < 20:
        start_hour_levels_train.append("evening")
    else:
        start_hour_levels_train.append("night")

start_hour_levels_test = []
for i in range(testset.shape[0]):
    if int(testset.iloc[i]['start_hour']) < 14:
        start_hour_levels_test.append("noon")
    elif int(testset.iloc[i]['start_hour']) < 17:
        start_hour_levels_test.append("afternoon")
    elif int(testset.iloc[i]['start_hour']) < 20:
        start_hour_levels_test.append("evening")
    else:
        start_hour_levels_test.append("night")

trainset['start_hour_label'] = start_hour_levels_train
testset['start_hour_label'] = start_hour_levels_test

In [7]:
# now we have 4 levels for start_hour, use get_dummies to convert it to one-hot encoding
trainset = pd.get_dummies(trainset, columns=['start_hour_label'])
testset = pd.get_dummies(testset, columns=['start_hour_label'])

In [8]:
trainset_columns = set(trainset.columns)
testset_columns = set(testset.columns)

# print the columns that are in trainset but not in testset
print(trainset_columns - testset_columns)

# print the columns that are in testset but not in trainset
print(testset_columns - trainset_columns)

set()
set()


In [150]:
testset['weather_Rain'] = 0
testset['weather_Drizzle'] = 0
testset['month_March'] = 0
testset['season'] = 2022

In [9]:
trainset.to_csv("processed_training.csv", index=False)
testset.to_csv("processed_test.csv", index=False)

In [153]:
# take a look at the original pickle
data_pickle = pd.read_pickle("../testset.pickle")
weather_description_list = []
for key, value in data_pickle.items():
    weather_description_list.append(value['weather_description'])

weather_description_list = list(set(weather_description_list))

# save the weather description list to a txt
with open("weather_description_list.txt", "w") as f:
    for weather_description in weather_description_list:
        f.write(weather_description + "\n")

In [155]:
weather_2022 = pd.read_csv("weather_2022.csv")

weather_2022.head()

,weather_Rain,weather_Drizzle
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [156]:
testset['weather_Rain'] = weather_2022['weather_Rain']
testset['weather_Drizzle'] = weather_2022['weather_Drizzle']

In [157]:
testset.to_csv("processed_test.csv", index=False)

In [158]:
testset['weather_Rain'].value_counts()

0    2437
1       8
Name: weather_Rain, dtype: int64

In [159]:
testset['weather_Drizzle'].value_counts()

0    2434
1      11
Name: weather_Drizzle, dtype: int64

##### add primary key

In [5]:
import pandas as pd
trainset = pd.read_csv("processed_training.csv")
testset = pd.read_csv("processed_test.csv")

# take a look at the data
print(f"trainset have {trainset.shape[0]} rows and {trainset.shape[1]} columns")
print(f"testset have {testset.shape[0]} rows and {testset.shape[1]} columns")

trainset have 12270 rows and 218 columns
testset have 2445 rows and 218 columns


In [2]:
url_train = pd.read_csv("game_page_url_train.csv")
url_test = pd.read_csv("game_page_url_test.csv")

In [3]:
trainset['game_page_url'] = url_train['game_page_url']
testset['game_page_url'] = url_test['game_page_url']
trainset.head()

,attendance,is_federal_holiday,venue,on_grass,temperature,wind_speed,team1_name,team1_pre_win,team1_pre_loss,team1_pre_win_pct,...,weather_Sunny,season_type,season,home_team_avg_att_last_year,start_hour,start_hour_label_afternoon,start_hour_label_evening,start_hour_label_night,start_hour_label_noon,game_page_url
0,35055,0,Wrigley Field,1,-2.786672,-0.093090,STL,-1.62645,-1.654514,0.0,...,1,regular,2015,0.345307,19,0,1,0,0,https://www.baseball-reference.com/boxes/CHN/C...
1,49043,0,Chase Field,1,0.705205,-0.093090,SFG,-1.62645,-1.654514,0.0,...,1,regular,2015,-0.573314,19,0,1,0,0,https://www.baseball-reference.com/boxes/ARI/A...
2,43633,0,Great American Ball Park,1,-0.521670,1.490839,PIT,-1.62645,-1.654514,0.0,...,0,regular,2015,0.066649,16,1,0,0,0,https://www.baseball-reference.com/boxes/CIN/C...
3,45030,0,Comerica Park,1,-2.031671,-0.489072,MIN,-1.62645,-1.654514,0.0,...,0,regular,2015,0.766348,13,0,0,0,1,https://www.baseball-reference.com/boxes/DET/D...
4,43753,0,Minute Maid Park,1,0.516455,1.292848,CLE,-1.62645,-1.654514,0.0,...,0,regular,2015,-1.084575,18,0,1,0,0,https://www.baseball-reference.com/boxes/HOU/H...


In [4]:
trainset.to_csv("processed_training.csv", index=False)
testset.to_csv("processed_test.csv", index=False)

In [6]:
start_time_train = pd.read_csv("start_time_train.csv")
start_time_test = pd.read_csv("start_time_test.csv")
trainset['start_time'] = start_time_train['start_time']
testset['start_time'] = start_time_test['start_time']
trainset.to_csv("processed_training.csv", index=False)
testset.to_csv("processed_test.csv", index=False)